In [86]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import functools
import numpy as np
from scipy.optimize import curve_fit
from ipywidgets import fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from IPython.display import clear_output
from IPython.display import display
from ipywidgets import Output

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [91]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, det_diam, pulse_mode, flightPathLength_d)

Dropdown(description='Data mode:', options=('PyCCAPT', 'leap_epos', 'leap_pos', 'ato_v6'), value='PyCCAPT')

FloatText(value=80.0, description='Detector diameter:')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

FloatText(value=110.0, description='Flight path length:')

$$\textbf{You can specify which dataset to use in below block}$$

In [131]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
variables.pulse_mode = pulse_mode
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path 
variables.result_data_name = 'mc_calibration_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/mc_calibration/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 11251


In [132]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: E:/APT&M-2023/physics_atom_probe_data/TiO2/data_TiO2
The dataset name after saving is: mc_calibration_cropped_data_TiO2
The figures will be saved on the path: E:/APT&M-2023/physics_atom_probe_data/TiO2/data_TiO2//mc_calibration/
Total number of Ions: 133038


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,0.0,104.964648,5300.0,0.0,0,0.0,2538.156982,-1.445110,-2.352110,0,0
1,0.0,0.0,0.0,0.0,19.805717,5300.0,0.0,0,0.0,1117.385864,3.482330,0.369837,0,0
2,0.0,0.0,0.0,0.0,20.773834,5300.0,0.0,0,0.0,1143.148560,2.394760,-2.384470,0,0
3,0.0,0.0,0.0,0.0,20.048524,5300.0,0.0,0,0.0,1118.774658,-0.524547,-2.450850,0,0
4,0.0,0.0,0.0,0.0,19.916877,5300.0,0.0,0,0.0,1118.870850,2.916350,-1.399770,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133033,0.0,0.0,0.0,0.0,127.338820,5300.0,0.0,0,0.0,2806.806885,-0.629373,3.616590,0,0
133034,0.0,0.0,0.0,0.0,96.286327,5300.0,0.0,0,0.0,2443.751221,-0.882558,3.604390,0,0
133035,0.0,0.0,0.0,0.0,104.046546,5300.0,0.0,0,0.0,2524.839600,-2.299070,-1.102040,0,0
133036,0.0,0.0,0.0,0.0,93.788365,5300.0,0.0,0,0.0,2403.310791,-0.037916,3.033020,0,0


In [133]:
interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=variables.max_tof), frac=widgets.FloatText(value=1.0),
                bins=fixed((1200,800)), figure_size=fixed((7,3)),
               draw_rect=fixed(False), data_crop=fixed(False), pulse_plot=widgets.Dropdown(options=[('False', False), ('True', True)]), dc_plot=widgets.Dropdown(options=[('True', True), ('False', False)]),
               pulse_mode=widgets.Dropdown(options=[('voltage', 'voltage'), ('laser', 'laser')]), save=widgets.Dropdown(options=[('True', True), ('False', False)]),
               figname=widgets.Text(value='exp_hist'));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_34656\1621577327.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(data_loadcrop.plot_crop_experiment_history, data=fixed(data), variables=fixed(variables), max_tof=widgets.FloatText(value=variables.max_tof), frac=widgets.FloatText(value=1.0),


interactive(children=(FloatText(value=11251.0, description='max_tof'), FloatText(value=1.0, description='frac'…

In [134]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc'), ('tof', 'tof')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=5000), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=fixed(False));

C:\Users\mehrp\AppData\Local\Temp\ipykernel_34656\2908251488.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

$$\textbf{Below plotted graph offers peak selection. You can select relevant peaks you want for computation.}$$

In [135]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=100, description='peak prominance:')
distance=widgets.IntText(value=500, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot peak:'
)

def hist_plot(variables, plot):
    
    with out:
        clear_output(True)
    
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.mc_hist_plot(variables, bin_size_p, mode='mc', prominence=prominence_p, distance=distance_p, percent=percent_p, selector='rect', plot=True, figname=index_fig_p, lim=lim_tof_p,
                          peaks_find_plot=plot_peak_p)

In [136]:
sample_size_b = widgets.IntText(value=11, description='sample size:') 
index_fig_b = widgets.IntText(value=1, description='fig index:')
maximum_cal_method_b = widgets.Dropdown(
    options=[('mean', 'mean'), ('histogram', 'histogram')],
    description='calib method:'
)
plot_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)

save_b = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
apply_b = widgets.Dropdown(
    options=[('all', 'all'), ('temporal', 'temporal'),],
    description='apply mode:'
)
def bowl_correction(dld_x, dld_y, dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_b.value
        index_fig_p = index_fig_b.value
        plot_p = plot_b.value
        save_p = save_b.value
        maximum_cal_method_p = maximum_cal_method_b.value
        calibration.bowl_correction_main(dld_x, dld_y, dld_highVoltage, variables, det_diam.value, sample_size=sample_size_p, maximum_cal_method=maximum_cal_method_p, apply_local=apply_b.value,
                                         calibration_mode='mc', index_fig=index_fig_p, plot=plot_p, save=save_p)


In [137]:
sample_size_v =widgets.IntText(value=100, description='sample size:') 
index_fig_v = widgets.IntText(value=1, description='fig index:')
plot_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='plot fig:'
)
save_v = widgets.Dropdown(
    options=[('False', False), ('True', True)],
    description='save fig:'
)
mode_v = widgets.Dropdown(
    options=[('ion_seq', 'ion_seq'), ('voltage', 'voltage')],
    description='sample mode:'
)
peak_mode = widgets.Dropdown(
    options=[('peak', 'peak'), ('mean', 'mean'), ('median', 'median')],
    description='peak mode:'
)
num_cluster = widgets.IntText(value=1, description='num_cluster:')
apply_v = widgets.Dropdown(
    options=[('all', 'all'), ('voltage', 'voltage'), ('voltage_temporal', 'voltage_temporal')],
    description='apply mode:'
)
def vol_correction(dld_highVoltage, variables):
    with out:
        sample_size_p = sample_size_v.value
        index_fig_p = index_fig_v.value
        plot_p = plot_v.value
        save_p = save_v.value
        mode_p = mode_v.value
        peak_mode_p = peak_mode.value
        calibration.voltage_corr_main(dld_highVoltage, variables, sample_size=sample_size_p, calibration_mode='mc',
                                        index_fig=index_fig_p, plot=plot_p, save=save_p, apply_local=apply_v.value, num_cluster=num_cluster.value, mode=mode_p, peak_mode=peak_mode_p)


In [138]:
pb_bowl = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
pb_vol = widgets.HTML(
    value=" ",
    placeholder='Status:',
    description='Status:',
)
plot_button = widgets.Button(
    description='plot hist',
)
plot_stat_button = widgets.Button(
    description='plot stat',
)
reset_back_button = widgets.Button(
    description='reset back correction',
)
reset_button = widgets.Button(
    description='reset',
)
save_button = widgets.Button(
    description='save correction',
)
bowl_button = widgets.Button(
    description='bowl correction',
)
vol_button = widgets.Button(
    description='voltage correction',
)
bin_fdm = widgets.IntText(value=256, description='bin FDM:')
    
@plot_button.on_click
def plot_on_click(b, variables=variables, plot=True):
    hist_plot(variables, plot)

@plot_stat_button.on_click
def plot_stat_on_click(b, variables=variables, save=True):
    with out2:
        clear_output(True)
    with out:
        clear_output(True)
        calibration.plot_selected_statistic(variables, bin_fdm.value, index_fig.value, calibration_mode='mc', save=True)
    
@reset_back_button.on_click
def reset_back_on_click(b, variables=variables):
    variables.mc_calib = np.copy(variables.mc_calib_backup)
@reset_button.on_click
def reset_on_click(b, data=data, variables=variables, flightPathLength_d=flightPathLength_d.value, max_mc=max_mc.value):
    data_tools.extract_data(data, variables, flightPathLength_d, max_mc)
    
@save_button.on_click
def save_on_click(b, variables=variables):
    variables.mc_calib_backup = np.copy(variables.mc_calib)

@vol_button.on_click
def vol_on_click(b, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_vol.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected mc ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            vol_correction(dld_highVoltage, variables)
        pb_vol.value = "<b>Flished</b>"
@bowl_button.on_click
def bowl_on_click(b, dld_x_det=variables.dld_x_det, dld_y_det=variables.dld_y_det, dld_highVoltage=variables.dld_high_voltage, variables=variables):
    with out2:
        clear_output(True)
        pb_bowl.value = "<b>Starting...</b>"
        if variables.selected_x1 == 0 or variables.selected_x2 == 0:
            print('Please first select a peak')
        else:
            print('Selected mc ranges are: (%s, %s)' %(variables.selected_x1, variables.selected_x2))
            bowl_correction(dld_x_det, dld_y_det, dld_highVoltage, variables)  
        pb_bowl.value = "<b>Flished</b>"


tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, bin_fdm, plot_peak])
tab2 = VBox(children=[VBox(children=[sample_size_b, index_fig_b, maximum_cal_method_b, apply_b, plot_b, save_b]), bowl_button, pb_bowl])
tab3 = VBox(children=[VBox(children=[sample_size_v, index_fig_v, mode_v, apply_v, num_cluster, peak_mode, plot_v, save_v]), vol_button, pb_vol])

tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'tof/mc plot')
tab.set_title(1, 'bowl correction')
tab.set_title(2, 'voltage correction')



display(VBox(children=[tab,HBox(children=[plot_button, plot_stat_button, save_button, reset_back_button, reset_button])]))
out = Output()
out2 = Output()
display(out)

Output()

In [139]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='mc_hist_calibrated'),
               peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_34656\419717795.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [126]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0], element[1], element[3]), "{}({})[{}]".format(element[0], element[1], element[2]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList,"Elements")
dropdown.observe(wd.on_change)


chargeDropdown = wd.dropdownWidget(chargeList,"Charge")
chargeDropdown.observe(wd.on_change_charge)

wd.compute_element_isotope_values_according_to_selected_charge()

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")


display(dropdown, chargeDropdown, buttonAdd, buttonDelete, buttonReset)

def buttonAdd_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickAdd(b, variables)
        display()
def buttonDelete_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickDelete(b, variables)
        display()
def buttonResett_f(b, variables):
    with out:
        clear_output(True)
        wd.onClickReset(b, variables)
        display()

listMaterial = buttonAdd.on_click(functools.partial(buttonAdd_f, variables=variables))
buttonDelete.on_click(functools.partial(buttonDelete_f, variables=variables))
buttonReset.on_click(functools.partial(buttonResett_f, variables=variables))
# listMaterial = buttonAdd.on_click(wd.onClickAdd)
# buttonDelete.on_click(wd.onClickDelete)
# buttonReset.on_click(wd.onClickReset)
out = Output()
display(out)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H(1)[1.01]'), ('H (2) (0.01)', 'H(2)[2.01]'), ('H…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

Output()

In [75]:
peaks_chos = []
for i in range(len(variables.peaks_idx)):
    peaks_chos.append(variables.peak[variables.peaks_idx[i]])
peaks_chos = np.array(peaks_chos)
listMaterial = np.array(variables.listMaterial)
print('highest peak in the mc histogram:', peaks_chos)
print('highest peak in the ideal mc histogram:', listMaterial)

highest peak in the mc histogram: [ 0.90012256 64.108729   96.31311392]
highest peak in the ideal mc histogram: [ 1.01 61.32 91.97]


In [78]:
def shift(mc, a, c):
    return  a * mc + c

# def shift(mc, a, b, c):
#     return a * ((mc - b)**2) + c

def shift_calib(mc, mc_ideal):
    
    fitresult, _ = curve_fit(shift, mc, mc_ideal, maxfev=2000)
    return fitresult

fitresult = shift_calib(peaks_chos,  variables.listMaterial)
variables.mc_calib = shift(variables.mc_calib_backup, *fitresult)

# from scipy import interpolate
# f_shift = interpolate.interp1d(np.array(variables.listMaterial).T, peaks_chos.T, fill_value="extrapolate")
# variables.mc_calib = f_shift(variables.mc_calib)



In [79]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc', 'mc')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=100), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('peak'), plot=fixed(True), figname=widgets.Text(value='mc_hist_calibrated'),
               peaks_find_plot=fixed(True));


C:\Users\mehrp\AppData\Local\Temp\ipykernel_34656\1203954349.py:1: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1),


interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [140]:
data['mc_c (Da)'] = variables.mc_calib
data

,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,0.0,0.0,0.0,104.114739,104.964648,5300.0,0.0,0,0.0,2538.156982,-1.445110,-2.352110,0,0
1,0.0,0.0,0.0,19.948638,19.805717,5300.0,0.0,0,0.0,1117.385864,3.482330,0.369837,0,0
2,0.0,0.0,0.0,20.739130,20.773834,5300.0,0.0,0,0.0,1143.148560,2.394760,-2.384470,0,0
3,0.0,0.0,0.0,19.909984,20.048524,5300.0,0.0,0,0.0,1118.774658,-0.524547,-2.450850,0,0
4,0.0,0.0,0.0,19.951156,19.916877,5300.0,0.0,0,0.0,1118.870850,2.916350,-1.399770,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133033,0.0,0.0,0.0,128.376545,127.338820,5300.0,0.0,0,0.0,2806.806885,-0.629373,3.616590,0,0
133034,0.0,0.0,0.0,97.030002,96.286327,5300.0,0.0,0,0.0,2443.751221,-0.882558,3.604390,0,0
133035,0.0,0.0,0.0,103.396403,104.046546,5300.0,0.0,0,0.0,2524.839600,-2.299070,-1.102040,0,0
133036,0.0,0.0,0.0,94.502242,93.788365,5300.0,0.0,0,0.0,2403.310791,-0.037916,3.033020,0,0


In [141]:
data_c = data.copy(deep = True)

In [129]:
# Remove negative mc
threshold = 0
mc_t = data_c['mc_c (Da)'].to_numpy()
mc_t_mask = (mc_t <= threshold)
print('The number of ions with negative mc are:', len(mc_t_mask[mc_t_mask==True]))
data_c.drop(np.where(mc_t_mask)[0], inplace=True)
data_c.reset_index(inplace=True, drop=True)

The number of ions with negative mc are: 54


In [142]:
interact_manual(data_tools.save_data, data=fixed(data_c), variables=fixed(variables),
                hdf=widgets.Dropdown(options=[('True', True), ('False', False)]),
                epos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                pos=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                ato_6v=widgets.Dropdown(options=[('False', False), ('True', True)]), 
                csv=widgets.Dropdown(options=[('False', False), ('True', True)]));

interactive(children=(Dropdown(description='hdf', options=(('True', True), ('False', False)), value=True), Dro…